In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_A2_024' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A2_024
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.003791382650379995
test_rmse: 0.003725249603701966
test_r2: 0.9218895516378692
feature number: 3
train_rmse: 0.003663628016848668
test_rmse: 0.0035241209410545233
test_r2: 0.9302351720188993
feature number: 4
train_rmse: 0.003489222998819841
test_rmse: 0.0034177720337545013
test_r2: 0.9344141176272135
feature number: 5
train_rmse: 0.0034519995251328533
test_rmse: 0.0034396534233453063
test_r2: 0.9331734458602459
feature number: 6
train_rmse: 0.003433021224928791
test_rmse: 0.00346078011809882
test_r2: 0.9324196793453755
feature number: 7
train_rmse: 0.0033738601841480573
test_rmse: 0.0034753908136802313
test_r2: 0.9319667526337048
feature number: 8
train_rmse: 0.0033386308410936483
test_rmse: 0.003558895603895498
test_r2: 0.9285479104622081
feature number: 9
train_rmse: 0.003315412858604579
test_rmse: 0.0035590922107732377
test_r2: 0.9284645568551664
feature number: 10
train_rmse: 0.003301455436617

train_rmse: 0.003095113228888705
test_rmse: 0.0041347875298897874
test_r2: 0.9020187948464315
feature number: 14
train_rmse: 0.0030811425573402415
test_rmse: 0.004195285415849796
test_r2: 0.8990550057479467
feature number: 15
train_rmse: 0.0030699310272105955
test_rmse: 0.004171637108321117
test_r2: 0.9000148167489213
feature number: 16
train_rmse: 0.0030405398029403814
test_rmse: 0.0040961124301291015
test_r2: 0.9036689272835254
feature number: 17
train_rmse: 0.003035397603764975
test_rmse: 0.004117512497636173
test_r2: 0.9027392773867682
feature number: 18
train_rmse: 0.0030312191361898605
test_rmse: 0.004138063686551314
test_r2: 0.9017482356971962
feature number: 19
train_rmse: 0.0030199734046978113
test_rmse: 0.004117933443072853
test_r2: 0.902781682175531
feature number: 20
train_rmse: 0.003015884910144634
test_rmse: 0.0041334785383404255
test_r2: 0.9020268408884753
Test_rmse_min = 1.fea_num: 4  2.rmse: 0.0034177720337545013
Test_r2_max = 1. fea_num: 4  2.r2: 0.9344141176272135
./

train_rmse: 0.0033738601841480573
test_rmse: 0.0034753908136802313
test_r2: 0.9319667526337048
feature number: 8
train_rmse: 0.0033386308410936483
test_rmse: 0.003558895603895498
test_r2: 0.9285479104622081
feature number: 9
train_rmse: 0.003315412858604579
test_rmse: 0.0035590922107732377
test_r2: 0.9284645568551664
feature number: 10
train_rmse: 0.003301455436617259
test_rmse: 0.0036801609259803137
test_r2: 0.9233807884185236
feature number: 11
train_rmse: 0.003261788296665769
test_rmse: 0.00392441737288722
test_r2: 0.9125936176875606
feature number: 12
train_rmse: 0.0032431759533159896
test_rmse: 0.003914581644750479
test_r2: 0.9131980723415534
feature number: 13
train_rmse: 0.003095113228888705
test_rmse: 0.0041347875298897874
test_r2: 0.9020187948464315
feature number: 14
train_rmse: 0.0030811425573402415
test_rmse: 0.004195285415849796
test_r2: 0.8990550057479467
feature number: 15
train_rmse: 0.0030699310272105955
test_rmse: 0.004171637108321117
test_r2: 0.9000148167489213
featu

# 可以修改第一層的神經元數量

In [8]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [9]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.007958949892429238
[0.005253559596404162, 0.005414501540226683, 0.002966404010949072, 0.013117204377355897, 0.003236166408485157, 0.003967098792310515, 0.02257348560114271, 0.01374309297704466, 0.0038155891983717755, 0.005502396422001764]


test_r2_ave: 0.4256379169686322
[0.8481990646295271, 0.8322013176583647, 0.9582394106486622, -0.09209734238198308, 0.9532678373420473, 0.9021500103324926, -1.5579910211110404, -0.3512567709459802, 0.9317734003819973, 0.8318932631322342]


feature number: 5
test_rmse_ave: 0.006084958410223331
[0.003129614047499252, 0.007430631964508707, 0.0034984410571686016, 0.0038456323929876497, 0.0036849364119193504, 0.004289835359442166, 0.013367867590578985, 0.0041641866480672095, 0.014327254629818797, 0.0031111840002425763]


test_r2_ave: 0.726631039081563
[0.9461297878292818, 0.6839738023353641, 0.9419161838553352, 0.9061327025132384, 0.9394081539354736, 0.8855815653920956, 0.1029306103838391, 0.8759407108253585, 0.038041216

In [17]:
fea_num=[4,5,6,7]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.006457012539497006
[0.0045792905807804105, 0.007847407866223047, 0.00307580742816256, 0.008483995491538908, 0.007933431246395015, 0.0016363762748701587, 0.0172557053359115, 0.007332070025246075, 0.003184260340144403, 0.003241780805697992]


test_r2_ave: 0.674771426809152
[0.8846643608779902, 0.6475284778969619, 0.9551022784095052, 0.5431436793365062, 0.7191487810106036, 0.9833512703546582, -0.49474530943042927, 0.6153886272593829, 0.9524831501743486, 0.9416489522019938]


feature number: 5
test_rmse_ave: 0.011509220040640953
[0.005331839409829083, 0.02852650122660329, 0.008081155047412424, 0.0060512548676621095, 0.00974134024345169, 0.0024147184858450163, 0.007001451355878972, 0.012629688162560037, 0.03202586614325657, 0.0032883854639103307]


test_r2_ave: -0.20698991602416378
[0.8436415912988977, -3.657668769926242, 0.6900776486086542, 0.7675820481610345, 0.5765602607052289, 0.9637466815216088, 0.7539190929875796, -0.1411800454215133, -3.806536821830

In [18]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.005653325060681482
[0.0043945241388838865, 0.004008745635527263, 0.009730936117175747, 0.0038953123407916897, 0.004237169460610916, 0.0033278863483701096, 0.00890309241589653, 0.009771779565565207, 0.005025837077539076, 0.003237967506454393]


test_r2_ave: 0.7849499463429995
[0.8937837829212678, 0.9080210212653212, 0.5506184243960084, 0.9036917806497569, 0.919886472374214, 0.9311424248400281, 0.6020912126079581, 0.3168497418569297, 0.8816284546510751, 0.9417861478674369]


feature number: 5
test_rmse_ave: 0.010057391046740981
[0.009932286699186586, 0.0057431251968243615, 0.02404487225708165, 0.004379407089838336, 0.010494630300881606, 0.0018551551680660273, 0.01039540310723325, 0.02072558007960739, 0.0069072692287939705, 0.006096181339896628]


test_r2_ave: 0.1844697644365102
[0.45741770853439223, 0.8112147103967359, -1.7437906257733036, 0.8782666947580586, 0.5085396600986307, 0.9786018989744318, 0.45751911870383577, -2.0731397010037544, 0.77641468613

In [19]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.006080563879977338
[0.002090025002818498, 0.007728788751683329, 0.0033689084584331876, 0.0078009468056325915, 0.009747931056612109, 0.009780998211779761, 0.007573850063467621, 0.004328540089533569, 0.005012109796390358, 0.003373540563422356]


test_r2_ave: 0.7572210973314564
[0.975974630024928, 0.6581036557832973, 0.9461377563328008, 0.6137456240736844, 0.5759870836591676, 0.40518609631733193, 0.7120380048754702, 0.8659546348330758, 0.8822741979779517, 0.936809289436856]


feature number: 5
test_rmse_ave: 0.005491068601121862
[0.004863003094381548, 0.006888816920277355, 0.004242035331686805, 0.006052507948484074, 0.003744639226225521, 0.008136027110173985, 0.00858657318863679, 0.005007533238315672, 0.003259232105974631, 0.004130317847062239]


test_r2_ave: 0.8112241206684635
[0.8699302777301312, 0.7283805478923163, 0.9146006465851158, 0.7674857809771138, 0.9374288476646246, 0.5884339652662329, 0.629880881357751, 0.8206024138547269, 0.9502192901577874,

In [20]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.008626033954693227
[0.008885054331152529, 0.01740076635139773, 0.010651637728849373, 0.0052182311188293, 0.003174118140670073, 0.0027590014053209063, 0.007162969323089196, 0.019369380502902666, 0.008816374511020707, 0.002822806133699778]


test_r2_ave: 0.36790256619353845
[0.5658025218420604, -0.7330389920781264, 0.46155816668031735, 0.8271675046027972, 0.9550426853132127, 0.9526719516592344, 0.7424343462091663, -1.6841104033063319, 0.6357407692798198, 0.955757111733235]


feature number: 5
test_rmse_ave: 0.007862964096998512
[0.011385488136601724, 0.01819754407048325, 0.0034740180411096566, 0.005322450650777993, 0.006735112124496257, 0.00607744746851838, 0.007108468453936168, 0.011689705553456288, 0.00510918261192064, 0.0035302238586847647]


test_r2_ave: 0.5299684788587553
[0.2870313542118943, -0.8953837051351272, 0.942724332576483, 0.8201948749263748, 0.7975844612378392, 0.7703549018021287, 0.74633891360827, 0.022366538122524315, 0.8776698927758552

In [21]:
fea_num=[4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.005028805113105606
[0.0036404023335478374, 0.002865670490083968, 0.009130114008575567, 0.003217440139473701, 0.0034559632923573088, 0.0021232164371369393, 0.006091948801493943, 0.0070279110404784274, 0.009613538155156644, 0.0031218464327517274]


test_r2_ave: 0.8310589862448575
[0.9271103459024664, 0.952997161484737, 0.6043980524290749, 0.9342948016186877, 0.9467042681333269, 0.9719712785296922, 0.8136993986475617, 0.6466367139653891, 0.566891193843537, 0.945886647894102]


feature number: 5
test_rmse_ave: 0.00729973004666266
[0.014636737087953324, 0.006932303561037044, 0.00553804253613322, 0.0029308558880357066, 0.011663479399611737, 0.00645462575457547, 0.008405832027283369, 0.009045731521897665, 0.0047041788762891106, 0.0026855138138099687]


test_r2_ave: 0.6396647959864615
[-0.1782994226637724, 0.7249404508286441, 0.8544478603904094, 0.9454785109345538, 0.39296959898431827, 0.7409659322484685, 0.6452983719419898, 0.4145951701640531, 0.896295366669

In [22]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.0060592077718071725
[0.003249556844969418, 0.004434035400503274, 0.003325805162742981, 0.014504854772831532, 0.0035091770788587425, 0.006195132996866386, 0.012031655048387876, 0.007295226484921574, 0.003117499578791177, 0.002929134349198763]


test_r2_ave: 0.6947306237744353
[0.9419214990744537, 0.8874695915532186, 0.9475072136517322, -0.3353820967701746, 0.945050372027266, 0.7613749562753113, 0.273304432233739, 0.6192442482335001, 0.954454726705777, 0.9523612947595301]


feature number: 5
test_rmse_ave: 0.007851886818433932
[0.00720450885301767, 0.0032804940389782712, 0.0030268759384153563, 0.010965259427515426, 0.004274000124971399, 0.009608358784547086, 0.010693540029807049, 0.010404792020342408, 0.007470885174518399, 0.011590153792226261]


test_r2_ave: 0.5834772726836078
[0.7145198739648345, 0.9384043434354834, 0.9565194266943018, 0.2368382381705857, 0.9184876808720043, 0.4259983037563031, 0.4259565495251306, 0.22547449958498034, 0.73843801269595

In [23]:
fea_num=[4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 4
test_rmse_ave: 0.006233435382594104
[0.0032870929937522616, 0.008062502922514585, 0.004380813016544073, 0.0030365989132911707, 0.0045625643107565445, 0.0015322327379867066, 0.006855023816174023, 0.008550973366363892, 0.01768736798457005, 0.004379183763987735]


test_r2_ave: 0.6979849254865056
[0.9405720016691901, 0.6279413952339381, 0.9089215872532517, 0.9414733529728168, 0.9071093104999838, 0.9854029791898803, 0.7641044744907961, 0.47688156955745853, -0.4660774985518439, 0.8935200825495847]


feature number: 5
test_rmse_ave: 0.010741754514537459
[0.01718510215209212, 0.018071904179513428, 0.0035071339682003918, 0.01134721114385967, 0.005550358840335792, 0.00620561782288661, 0.005792726351716591, 0.03272185323098652, 0.002955310016269706, 0.0040803274395137595]


test_r2_ave: -0.270824911943835
[-0.6243191225630507, -0.8693017536505356, 0.9416271724092643, 0.1827460056446477, 0.8625336782107103, 0.7605665606366363, 0.8315512497994346, -6.660280955561526, 0.95907048514